In [2]:
# Installing necessary library
# ! pip install flair

     |████████████████████████████████| 322 kB 5.0 MB/s 
     |████████████████████████████████| 59 kB 5.6 MB/s 
     |████████████████████████████████| 981 kB 31.7 MB/s 
     |████████████████████████████████| 64 kB 2.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.1 MB 41.5 MB/s 
     |████████████████████████████████| 19.7 MB 1.4 MB/s 
     |████████████████████████████████| 1.2 MB 51.0 MB/s 
     |████████████████████████████████| 788 kB 76.0 MB/s 
     |████████████████████████████████| 48 kB 4.9 MB/s 
     |████████████████████████████████| 62 kB 762 kB/s 
     |████████████████████████████████| 895 kB 55.3 MB/s 
     |████████████████████████████████| 596 kB 55.3 MB/s 
     |████████████████████████████████| 3.3 MB 34.7 MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-py3-none-any.whl size=9704 sha256=03b11ad12db4fcd9de344023b4c40afda9cb5a32dad07d

In [16]:
# Importing necessary libraries
from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
tagger = SequenceTagger.load("flair/ner-english-ontonotes")

2021-11-21 12:46:28,563 loading file /root/.flair/models/ner-english-ontonotes/f46dcd14689a594a7dd2a8c9c001a34fd55b02fded2528410913c7e88dbe43d4.1207747bf5ae24291205b6f3e7417c8bedd5c32cacfb5a439f3eff38afda66f7
Sentence: "On September 1st 2011 George Washington won 1 dollar ."   [− Tokens: 10  − Token-Labels: "On September <B-DATE> 1st <I-DATE> 2011 <E-DATE> George <B-PERSON> Washington <E-PERSON> won 1 <B-MONEY> dollar <E-MONEY> ."]
The following NER tags are found:
Span [2,3,4]: "September 1st 2011"   [− Labels: DATE (0.9475)]
Span [5,6]: "George Washington"   [− Labels: PERSON (0.9334)]
Span [8,9]: "1 dollar"   [− Labels: MONEY (0.9861)]


In [5]:
import requests
from bs4 import BeautifulSoup

# site to scrape
URL = "https://thebestschools.org/magazine/most-influential-computer-scientists/"
response = requests.get(URL)
soup = BeautifulSoup(response.text, "html.parser")

In [6]:
job_elements = soup.select('section.container.content.pt-3.pb-8.js-content')[0].select('section')

print(job_elements[57])

<section>
<h3 id="wootters"><span> </span> William K. Wootters</h3>
<p class="italic">Quantum information theory, No-cloning theorem, Quantum teleportation, Quantum cryptography</p>
<aside class="float-right" style="width:180px;"><img alt="Image of William K. Wootters" class="float-right" loading="lazy" src="https://res.cloudinary.com/highereducation/image/upload/h_300,w_180,c_scale,f_auto,q_auto:eco,/v1/TheBestSchools.org/william-wootters" style="width:180px;height:300px;"/> </aside>
<p>Wootters was probably born around 1951, given that he obtained his bachelor’s degree in 1973 from Stanford University. He earned his Ph.D. in physics in 1980 from the University of Texas at Austin, writing his dissertation on quantum information theory. During postdoctoral research at that school, Wootters worked with famed physicist John Archibald Wheeler. He is currently Barclay Jermain Professor of Natural Philosophy, Emeritus, in the Physics Department at Williams College.</p>
<p>Wootters is known 

In [53]:
names = []
AOI = []
img_links = []
weblinks = []
book_names = []
bookLinks = []
content = []
place_of_birth = []
year_of_birth = []


for job in job_elements[1:58]:
    cs_name=""
    imgLink=""
    all_interest=[]
    website=""
    bookLink=""
    book_name=""
    data = ""
    book = ""
    
    try:
        cs_name = job.select('h3')[0].getText()
        names.append((cs_name[2:]))
    except Exception:
        names.append("")
    
    try:
        imgLink = job.select('aside')[0].select('img')[0].get('src')
    except Exception:
        pass
    img_links.append(imgLink)
    
    try:
        interests = job.select('p')[0].getText()
        all_interest = interests.split(',')
    except Exception:
        pass
    AOI.append(all_interest)
    
    try:
        website = job.select('aside')[1].select('p')[0].select('a')[0].get('href')
    except Exception:
        pass
    weblinks.append(website)
    
    try:
        bookLink = job.select('aside')[1].select('p')[1].select('a')[0].get('href')
    except Exception:
        pass
    bookLinks.append(bookLink)
    
    try:
        book_name = job.select('ul')[0].select('li')
        l = len(book_name)
        book = []
        for i in range(l):
            book.append(book_name[i].getText())

    except Exception as e:
        pass
    book_names.append(book)
    first_line = ""
    try:
        data = job.select('p')
        data1 = data[1].getText()
        first_line = data1.split(".")[0]
        data1 += (" ") + (data[2].getText())
    except Exception:
        pass
    places = ""
    date = ""
      
    # getting dob and place of birth from the first line of first paragraph, using pre-trained flair model.
    if first_line!="":
      sentence = Sentence(first_line)

      # predict NER tags
      tagger.predict(sentence)
      entities = sentence.to_dict(tag_type='ner')['entities']
      allowed_labels = ["DATE", "GPE"]
      for entity in entities:
        
        label = str(entity["labels"][0])
        label = label.split()[0]
        
        if label=="DATE":
          date = entity["text"]
        elif label=="GPE":
          text = entity["text"]
          places += (text + ",")
      
    place_of_birth.append(places[:-1])
    year_of_birth.append(date)
    content.append(data1)

In [54]:
# converting data into pandas dataframe

import pandas as pd
df = pd.DataFrame(list(zip(names,AOI,img_links,weblinks,book_names,bookLinks,content, place_of_birth, year_of_birth)),columns =["names","AOI","img_links","weblinks","book_names","bookLinks","content", "place_of_birth", "year_of_birth"])

In [55]:
(df.head(5))

,names,AOI,img_links,weblinks,book_names,bookLinks,content,place_of_birth,year_of_birth
0,Leonard M. Adleman,"[Computational Complexity Theory, Cryptography]",https://res.cloudinary.com/highereducation/ima...,https://viterbi.usc.edu/directory/faculty/Adle...,[Primality Testing and Abelian Varieties Over ...,https://www.amazon.com/Leonard-M.-Adleman/e/B0...,"Adleman was born in San Francisco, California,...","San Francisco,California",1945
1,Frances E. Allen,"[Compilers, Program optimization, Parallel c...",https://res.cloudinary.com/highereducation/ima...,https://www-03.ibm.com/ibm/history/witexhibit/...,,,"Allen was born in the town of Peru, in upstate...","Peru,New York",1932
2,Timothy J. Berners-Lee,"[Network design, World Wide Web, HTTP]",https://res.cloudinary.com/highereducation/ima...,https://www.cs.ox.ac.uk/people/tim.berners-lee/,[Weaving the Web: The Original Design and Ulti...,https://www.csail.mit.edu/person/tim-berners-lee,Berners-Lee was born in London in 1955. He rec...,London,1955
3,Manuel Blum,"[Computational Complexity Theory, Blum-Goldwa...",https://res.cloudinary.com/highereducation/ima...,https://www.cs.cmu.edu/~mblum/,"[Cryptographic Techniques & E-Commerce, co-edi...",https://www.amazon.com/s/ref=nb_sb_noss?url=se...,"Blum was born in Caracas, Venezuela, in 1938. ...","Caracas,Venezuela",1938
4,Kathleen Booth,"[Assembly language, ARC-2]",,,"[Automatic Digital Calculators, co-authored wi...",,"Booth (née Britten) was born in Stourbridge, U...","Stourbridge,UK",1922


In [56]:
# df.to_csv("output.csv", index=False)

In [57]:
# converting pandas dataframe to json

import json

json_obj = {}
img_links,weblinks,book_names,bookLinks,content
n_rows = df.shape[0]
for i in range(n_rows):
    
    # setting key as scientist's name
    key = df.names[i].lower()
    
    json_obj[key]={}
    json_obj[key]["name"] = df.names[i]
    json_obj[key]["area of interest"] = df.AOI[i]
    json_obj[key]["Image Link"] = df.img_links[i]
    json_obj[key]["Website"] = df.weblinks[i]
    json_obj[key]["Book Name"] = df.book_names[i]
    json_obj[key]["Book Link"] = df.bookLinks[i]
    json_obj[key]["Content"] = df.content[i]
    json_obj[key]["place_of_birth"] = df.place_of_birth[i]
    json_obj[key]["year_of_birth"] = df.year_of_birth[i]

In [58]:
# creating json object

json_final_obj = json.dumps(json_obj, indent=4, ensure_ascii=False)
json_final_obj

'{\n    "leonard m. adleman": {\n        "name": "Leonard M. Adleman",\n        "area of interest": [\n            "Computational Complexity Theory",\n            " Cryptography"\n        ],\n        "Image Link": "https://res.cloudinary.com/highereducation/image/upload/h_300,w_180,c_scale,f_auto,q_auto:eco,/v1/TheBestSchools.org/leonard-adleman",\n        "Website": "https://viterbi.usc.edu/directory/faculty/Adleman/Leonard",\n        "Book Name": [\n            "Primality Testing and Abelian Varieties Over Finite Fields, co-authored with Ming-Deh Huang (Springer, 1992)",\n            "Algorithmic Number Theory, co-edited with Ming-Deh Huang (Springer, 1994)"\n        ],\n        "Book Link": "https://www.amazon.com/Leonard-M.-Adleman/e/B00JDA6YGC/?tag=tbs242-20",\n        "Content": "Adleman was born in San Francisco, California, in 1945. He received his Ph.D. in electrical engineering and computer sciences (EECS) from UC-Berkeley in 1976. He is currently Distinguished Professor of C

In [59]:
# dumping json object to a file

f = open("MostInfluentialComputerScientists.json","w",encoding="utf-8")
f.write(json_final_obj)
f.close()